<a href="https://colab.research.google.com/github/sudha702560/MiniCPM_GGUF/blob/main/MiniCPM_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.15-cp311-cp311-linux_x86_64.whl size=4397835 sha256=c6cfd0542f0ea27c6cf8a0299dce858eba46e2583600881c606d842c03a01f12
  Stored in directory: /root/.cache/pip/wheels/34/62/22/63039c8c4c8da7ff68e81b8357e64c199faff9df9a2b5e5e3b
Successfully built llama-cpp-python


In [ ]:
!pip install llama-cpp-python huggingface_hub pandas --quiet

In [ ]:
import pandas as pd
import time
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import re

In [ ]:
# df1 = pd.read_csv('/content/dataset.csv', encoding='ISO-8859-1')
# df1 = df1.dropna()
# df1 = df1.drop_duplicates()
# df1.reset_index(drop=True, inplace=True)
# df = df1[['complaint_text']].sample(n=100, random_state=42).reset_index(drop=True)
# df['complaint_text'] = df['complaint_text'].apply(
#     lambda x: re.sub(r'[^\x00-\x7F]+', ' ', str(x))
# )
# df['complaint_text'] = df['complaint_text'].str.replace('?', '', regex=False)
# # df1=df1.head(100)
# df

In [ ]:
df = pd.read_excel('/content/DataSet_Complaint_Text_GPT-5.xlsx');
df.head(10)

,Complaint,Sentiment,Department,Severity
0,"Deposited crossed cheque, paid in cash.",Neutral,Cheque Services,1
1,I'm getting duplicate alerts for the same NEFT...,Neutral,SMS & Alerts,1
2,My replacement card hasn't arrived even after ...,Neutral,Debit Card / Credit Card,1
3,I was charged 50 for every mini statement. To...,Negative,Charges & Fees,1
4,The app doesn't load transactions beyond 15 days.,Neutral,Mobile Banking,1
5,The machine said temporary technical issue ...,Negative,Customer Service / Call Center,1
6,Charged 50 for balance inquiry SMS.,Negative,SMS & Alerts,1
7,The mobile app is extremely slow and freezes w...,Negative,Mobile Banking,3
8,They refused to accept a PDF KYC. Must be phys...,Neutral,Account Management,1
9,The app crashes every time I try to view my tr...,Neutral,Mobile Banking,1


In [ ]:
departments_list = sorted(df['Department'].unique())
departments_list

['ATM Services',
 'Account Management',
 'Charges & Fees',
 'Cheque Services',
 'Customer Service / Call Center',
 'Debit Card / Credit Card',
 'Deposits & Withdrawals',
 'Digital Wallets / UPI',
 'Insurance (Bank-Sold)',
 'Internet Banking',
 'Loans (Personal, Home, Auto)',
 'Mobile Banking',
 'SMS & Alerts',
 'Wealth Management / Investment']

In [ ]:
print(f"""You are an AI assistant for a bank's complaint classification system.
Your task is to carefully read the given complaint and classify it into exactly three categories:

1. Sentiment:
   - Positive, Neutral, or Negative
   - Base this on the customer's tone and overall satisfaction level.

2. Department:
   - Choose only ONE department from this exact list (no extra words, no variations): {', '.join(departments_list)})
   - If multiple departments seem relevant, choose the one that is most directly responsible.

3. Severity:
   - Choose only 1, 3, or 5
     - 1 = Low (minor inconvenience, no immediate impact)
     - 3 = Medium (moderate issue affecting some transactions or causing delays)
     - 5 = High (serious problem such as fraud, financial loss, or service unavailability)

Output Format (strict, no extra text):
Sentiment: <Positive/Neutral/Negative>
Department: <one of the departments from the list above>
Severity: <1/3/5>

Complaint:
Answer:
""")

You are an AI assistant for a bank's complaint classification system.
Your task is to carefully read the given complaint and classify it into exactly three categories:

1. Sentiment:
   - Positive, Neutral, or Negative
   - Base this on the customer's tone and overall satisfaction level.

2. Department:
   - Choose only ONE department from this exact list (no extra words, no variations): ATM Services, Account Management, Charges & Fees, Cheque Services, Customer Service / Call Center, Debit Card / Credit Card, Deposits & Withdrawals, Digital Wallets / UPI, Insurance (Bank-Sold), Internet Banking, Loans (Personal, Home, Auto), Mobile Banking, SMS & Alerts, Wealth Management / Investment)
   - If multiple departments seem relevant, choose the one that is most directly responsible.

3. Severity:
   - Choose only 1, 3, or 5
     - 1 = Low (minor inconvenience, no immediate impact)
     - 3 = Medium (moderate issue affecting some transactions or causing delays)
     - 5 = High (serious prob

In [ ]:
def build_prompt(complaint):
    return f"""You are an AI assistant for a bank's complaint classification system.
Your task is to carefully read the given complaint and classify it into exactly three categories:

1. Sentiment:
   - Positive, Neutral, or Negative
   - Base this on the customer's tone and overall satisfaction level.

2. Department:
   - Choose only ONE department from this exact list (no extra words, no variations): {', '.join(departments_list)})
   - If multiple departments seem relevant, choose the one that is most directly responsible.

3. Severity:
   - Choose only 1, 3, or 5
     - 1 = Low (minor inconvenience, no immediate impact)
     - 3 = Medium (moderate issue affecting some transactions or causing delays)
     - 5 = High (serious problem such as fraud, financial loss, or service unavailability)

Output Format (strict, no extra text):
Provide the output in JSON format.
{{
  "Sentiment": "<Positive/Neutral/Negative>",
  "Department": "<one of the departments from the list above>",
  "Severity": "<1/3/5>"
}}

Complaint:  {complaint}
Answer:
""".strip()

#Q8_0

In [ ]:
REPO_ID = "bartowski/MiniCPM-o-2_6-GGUF"
MODEL_FILE = "MiniCPM-o-2_6-Q8_0.gguf"

model_path = hf_hub_download(repo_id=REPO_ID, filename=MODEL_FILE)

llm = Llama(
    model_path=model_path,
    n_threads=8,   # adjust to your Colab CPU cores
    n_ctx=2048,
    verbose=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


MiniCPM-o-2_6-Q8_0.gguf:   0%|          | 0.00/8.10G [00:00<?, ?B/s]

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [ ]:
# import re
# import json

# def extract_values(response):
#     try:
#         # Assuming the response is now in JSON format
#         data = json.loads(response.strip())
#         sentiment = data.get("Sentiment")
#         department = data.get("Department")
#         severity = data.get("Severity")
#         return sentiment, department, severity
#     except json.JSONDecodeError:
#         # Fallback to regex if JSON parsing fails
#         text = response.strip()
#         sentiment_match = re.search(r"sentiment\s*[:\-]\s*(positive|neutral|negative)", text, re.I)
#         department_match = re.search(r"department\s*[:\-]\s*([A-Za-z &]+)", text, re.I)
#         severity_match = re.search(r"[sS]everity\s*[:\-]\s*([1-5])", text, re.I)

#         sentiment = sentiment_match.group(1).capitalize() if sentiment_match else None
#         department = department_match.group(1).strip() if department_match else None
#         severity = severity_match.group(1) if severity_match else None

#         return sentiment, department, severity
import re
import json

def extract_values(response):
    try:
        # Try to parse as JSON
        data = json.loads(response.strip())

        # Ensure it's a dictionary before accessing keys
        if isinstance(data, dict):
            sentiment = data.get("Sentiment")
            department = data.get("Department")
            severity = data.get("Severity")
            return sentiment, department, severity
        else:
            # Not a dict → fallback to regex
            raise ValueError("Parsed JSON is not a dictionary")

    except (json.JSONDecodeError, ValueError, AttributeError):
        # Fallback to regex if JSON parsing fails or wrong type
        text = response.strip()
        sentiment_match = re.search(r"sentiment\s*[:\-]\s*(positive|neutral|negative)", text, re.I)
        department_match = re.search(r"department\s*[:\-]\s*([A-Za-z &/()]+)", text, re.I)
        severity_match = re.search(r"[sS]everity\s*[:\-]\s*([1-5])", text, re.I)

        sentiment = sentiment_match.group(1).capitalize() if sentiment_match else None
        department = department_match.group(1).strip() if department_match else None
        severity = severity_match.group(1) if severity_match else None

        return sentiment, department, severity


In [ ]:
import time
def predict(complaint):
    prompt = build_prompt(complaint)
    # print(prompt)
    start_time = time.time()
    result = llm(
        prompt,
        max_tokens=50,
        temperature=0,     # deterministic output
        stop=["\n\n", "</s>"]
    )
    elapsed = time.time() - start_time
    response = result["choices"][0]["text"]
    # print(response)
    sentiment, department, severity = extract_values(response)

    return sentiment, department, severity, elapsed


In [ ]:
sentiments, departments, severities, ti = [], [], [], []
i=1
for complaint in df['Complaint']:
    s, d, v, t = predict(complaint)
    print("OUTPUT ",i," : ", s, d, v, t)
    sentiments.append(s)
    departments.append(d)
    severities.append(v)
    ti.append(round(t,2))
    i+=1

OUTPUT  1  :  Neutral Deposits & Withdrawals 1 24.04463768005371
OUTPUT  2  :  Neutral SMS & Alerts 3 22.506750345230103
OUTPUT  3  :  Neutral Debit Card / Credit Card 3 25.73669195175171
OUTPUT  4  :  Negative Charges & Fees 3 28.066426515579224
OUTPUT  5  :  Neutral Deposits & Withdrawals 3 30.07343602180481
OUTPUT  6  :  Neutral ATM Services 3 23.731165647506714
OUTPUT  7  :  Negative Charges & Fees 3 21.521671295166016
OUTPUT  8  :  Negative Mobile Banking 3 22.420366764068604
OUTPUT  9  :  Negative Customer Service / Call Center 3 27.615097284317017
OUTPUT  10  :  Neutral Mobile Banking 3 22.128287315368652
OUTPUT  11  :  Negative Charges & Fees 3 22.64241361618042
OUTPUT  12  :  Neutral Deposits & Withdrawals 3 28.82707929611206
OUTPUT  13  :  Neutral Digital Wallets / UPI 3 25.003055572509766
OUTPUT  14  :  Negative SMS & Alerts 3 22.728441953659058
OUTPUT  15  :  Neutral ATM Services 3 19.833221673965454
OUTPUT  16  :  Negative Cheque Services 3 75.35109043121338
OUTPUT  17  : 

In [ ]:
df['p_sentiment_Q8_0'] = sentiments
df['p_department_Q8_0'] = departments
df['p_severity_Q8_0'] = severities
df['p_time_taken_Q8_0'] = ti

In [ ]:
df

,Complaint,Sentiment,Department,Severity,p_sentiment_Q8_0,p_department_Q8_0,p_severity_Q8_0,p_time_taken_Q8_0
0,"Deposited crossed cheque, paid in cash.",Neutral,Cheque Services,1,Neutral,Deposits & Withdrawals,1,24.04
1,I'm getting duplicate alerts for the same NEFT...,Neutral,SMS & Alerts,1,Neutral,SMS & Alerts,3,22.51
2,My replacement card hasn't arrived even after ...,Neutral,Debit Card / Credit Card,1,Neutral,Debit Card / Credit Card,3,25.74
3,I was charged 50 for every mini statement. To...,Negative,Charges & Fees,1,Negative,Charges & Fees,3,28.07
4,The app doesn't load transactions beyond 15 days.,Neutral,Mobile Banking,1,Neutral,Deposits & Withdrawals,3,30.07
...,...,...,...,...,...,...,...,...
95,"I was promised a 9% interest rate, but now it ...",Negative,Customer Service / Call Center,1,Negative,"Loans (Personal, Home, Auto)",3,29.25
96,The agent I spoke to didn t understand my issu...,Negative,Customer Service / Call Center,1,Negative,Customer Service / Call Center,3,25.45
97,The app crashes on startup after the latest up...,Neutral,Mobile Banking,1,Neutral,Mobile Banking,3,19.97
98,The ATM camera was not working. My transaction...,Negative,ATM Services,1,Negative,ATM Services,3,26.72


#Q8_0 Evaluation

In [ ]:
se=sum(ti)
sec = se%60
mn = se/60
print(round(mn,0)," min and ",round(sec,0)," sec")

44.0  min and  17.0  sec


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
y_true = df['Sentiment'].fillna("").str.lower()
y_pred = df['p_sentiment_Q8_0'].fillna("").str.lower()
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Accuracy:  0.67
Precision: 0.45
Recall:    0.44
F1 Score:  0.45


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_true = df['Department'].fillna("").str.lower()
y_pred = df['p_department_Q8_0'].fillna("").str.lower()
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Accuracy:  0.61
Precision: 0.62
Recall:    0.56
F1 Score:  0.53


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# y_true = df1['Actual_severity'].fillna("").str.lower()
y_true = df['Severity'].fillna("").apply(lambda x: str(x).lower())
y_pred = df['p_severity_Q8_0'].fillna("").apply(lambda x: str(x).lower())
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Accuracy:  0.18
Precision: 0.37
Recall:    0.31
F1 Score:  0.14


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Sentimental_Analysis/MiniCPM_GGUF_2.xlsx'

# Save DataFrame to Excel
df.to_excel(file_path, index=False)